### Import Spark Session

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

### Bring In Dataset

In [2]:
df = spark.read.csv('ECdata.csv',header=True,inferSchema=True)
df.limit(5)

Brand,Model,Accel,TopSpeed,Range,Efficiency,FastCharge,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
Tesla,Model 3 Long Rang...,4.6 sec,233 km/h,450 km,161 Wh/km,940 km/h,Rapid charging po...,All Wheel Drive,Type 2 CCS,Sedan,D,5,55480
Volkswagen,ID.3 Pure,10.0 sec,160 km/h,270 km,167 Wh/km,250 km/h,Rapid charging po...,Rear Wheel Drive,Type 2 CCS,Hatchback,C,5,30000
Polestar,2,4.7 sec,210 km/h,400 km,181 Wh/km,620 km/h,Rapid charging po...,All Wheel Drive,Type 2 CCS,Liftback,D,5,56440
BMW,iX3,6.8 sec,180 km/h,360 km,206 Wh/km,560 km/h,Rapid charging po...,Rear Wheel Drive,Type 2 CCS,SUV,D,5,68040
Honda,e,9.5 sec,145 km/h,170 km,168 Wh/km,190 km/h,Rapid charging po...,Rear Wheel Drive,Type 2 CCS,Hatchback,B,4,32997


### Rename Column Headers

In [3]:
col_names = ['Brand', 'Model','AccelSec','TopSpeed_Km','Range_Km','Efficiency_Wh/km','FastCharge_km/h','RapidCharge','PowerTrain','PlugType','Body','Segment','Seats','PriceEuro']
df = df.toDF(*col_names)

In [4]:
df.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- AccelSec: string (nullable = true)
 |-- TopSpeed_Km: string (nullable = true)
 |-- Range_Km: string (nullable = true)
 |-- Efficiency_Wh/km: string (nullable = true)
 |-- FastCharge_km/h: string (nullable = true)
 |-- RapidCharge: string (nullable = true)
 |-- PowerTrain: string (nullable = true)
 |-- PlugType: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- PriceEuro: integer (nullable = true)



### Import SQL functions

In [6]:
from pyspark.sql.types import IntegerType,FloatType
from pyspark.sql.functions import col, regexp_replace, when,countDistinct

### Remove strings from Integer Column Types
##### 'FastCharge' contained hyphen values representing NULL and ruined 2 hours of my time trying to figure out why I was only getting one distinct value for 'RapidCharge'. Youll notice the regex pattern contains hyphen.

In [7]:
df = df.withColumn("AccelSec",regexp_replace(col("AccelSec"),"[^0-9.]",""))\
    .withColumn("TopSpeed_Km",regexp_replace(col("TopSpeed_Km"),"[^0-9.]",""))\
    .withColumn("Range_Km",regexp_replace(col("Range_Km"),"[^0-9.]",""))\
    .withColumn("Efficiency_Wh/km",regexp_replace(col("Efficiency_Wh/km"),"[^0-9.]",""))\
    .withColumn("FastCharge_km/h",regexp_replace(col("FastCharge_km/h"),"/[^\d-]/g",""))

df.limit(5)

Brand,Model,AccelSec,TopSpeed_Km,Range_Km,Efficiency_Wh/km,FastCharge_km/h,RapidCharge,PowerTrain,PlugType,Body,Segment,Seats,PriceEuro
Tesla,Model 3 Long Rang...,4.6,233,450,161,940 km/h,Rapid charging po...,All Wheel Drive,Type 2 CCS,Sedan,D,5,55480
Volkswagen,ID.3 Pure,10.0,160,270,167,250 km/h,Rapid charging po...,Rear Wheel Drive,Type 2 CCS,Hatchback,C,5,30000
Polestar,2,4.7,210,400,181,620 km/h,Rapid charging po...,All Wheel Drive,Type 2 CCS,Liftback,D,5,56440
BMW,iX3,6.8,180,360,206,560 km/h,Rapid charging po...,Rear Wheel Drive,Type 2 CCS,SUV,D,5,68040
Honda,e,9.5,145,170,168,190 km/h,Rapid charging po...,Rear Wheel Drive,Type 2 CCS,Hatchback,B,4,32997


### Alter Table Columns

In [8]:
df = df.withColumn("AccelSec",col("AccelSec").cast(FloatType()))\
    .withColumn("TopSpeed_Km",col("TopSpeed_Km").cast(IntegerType()))\
    .withColumn("Range_Km",col("Range_Km").cast(IntegerType()))\
    .withColumn("Efficiency_Wh/km",col("Efficiency_Wh/km").cast(IntegerType()))\
    .withColumn("FastCharge_km/h",col("FastCharge_km/h").cast(IntegerType()))\
    .withColumn("Seats",col("Seats").cast(IntegerType()))\
    .withColumn("PriceEuro",col("PriceEuro").cast(IntegerType()))

df.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- AccelSec: float (nullable = true)
 |-- TopSpeed_Km: integer (nullable = true)
 |-- Range_Km: integer (nullable = true)
 |-- Efficiency_Wh/km: integer (nullable = true)
 |-- FastCharge_km/h: integer (nullable = true)
 |-- RapidCharge: string (nullable = true)
 |-- PowerTrain: string (nullable = true)
 |-- PlugType: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- PriceEuro: integer (nullable = true)



### Validate Column

In [9]:
df = df.withColumn('RapidCharge',when(col('RapidCharge').like('%not%'),'No').otherwise('Yes'))
df.select(countDistinct('RapidCharge'))

count(DISTINCT RapidCharge)
2
